Doc: https://huggingface.co/datasets/PolyAI/minds14

In [5]:
from datasets import load_dataset

nome_dataset = "PolyAI/minds14"
idioma_dataset = "pt-PT"

dados = load_dataset(
    nome_dataset,
    name=idioma_dataset,
    split='train',
    streaming=True,
    trust_remote_code=True  # <- Adicionado aqui
)
dados

IterableDataset({
    features: ['path', 'audio', 'transcription', 'english_transcription', 'intent_class', 'lang_id'],
    num_shards: 1
})

In [8]:
import IPython

for linha in dados.take(5):
    audio = linha['audio']['array']
    taxa_amostragem = linha['audio']['sampling_rate']
    display(IPython.display.Audio(data=audio, rate=taxa_amostragem))

Doc: https://huggingface.co/openai/whisper-medium

In [9]:
from transformers import pipeline

modelo = 'openai/whisper-medium'

reconhecedor = pipeline('automatic-speech-recognition', model=modelo)

config.json: 0.00B [00:00, ?B/s]

C:\Users\Master\Desktop\IA-imagens-audio\venv\Lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Master\.cache\huggingface\hub\models--openai--whisper-medium. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to reg

model.safetensors:   0%|          | 0.00/3.06G [00:00<?, ?B/s]

generation_config.json: 0.00B [00:00, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

...

normalizer.json: 0.00B [00:00, ?B/s]

added_tokens.json: 0.00B [00:00, ?B/s]

special_tokens_map.json: 0.00B [00:00, ?B/s]

preprocessor_config.json: 0.00B [00:00, ?B/s]

Device set to use cpu


In [14]:
reconhecedor.feature_extractor.sampling_rate

16000

In [17]:
dados

IterableDataset({
    features: ['path', 'audio', 'transcription', 'english_transcription', 'intent_class', 'lang_id'],
    num_shards: 1
})

In [19]:
reconhecedor(next(iter(dados))["audio"])

C:\Users\Master\Desktop\IA-imagens-audio\venv\Lib\site-packages\transformers\models\whisper\generation_whisper.py:604: FutureWarning: The input name `inputs` is deprecated. Please make sure to use `input_features` instead.
  warnings.warn(
Using custom `forced_decoder_ids` from the (generation) config. This is deprecated in favor of the `task` and `language` flags/config options.
Transcription using a multilingual Whisper will default to language detection followed by transcription instead of translation to English. This might be a breaking change for your use case. If you want to instead always translate your audio to English, make sure to pass `language='en'`. See https://github.com/huggingface/transformers/pull/28687 for more details.


{'text': ' Bom dia, estou a ligar porque precisava de informações sobre como eu posso depositar dinheiro na minha conta.'}

In [29]:
from transformers import pipeline

modelo = 'openai/whisper-medium'

reconhecedor = pipeline(
    'automatic-speech-recognition', 
    model=modelo,
    generate_kwargs= {"task" : "translate", "language": "portuguese" },
)

Device set to use cpu


In [30]:
from itertools import islice

audio = next(islice(dados, 7, 8))
display(IPython.display.Audio(data=audio['audio']['array'], rate=audio['audio']['sampling_rate']))

In [28]:
reconhecedor(audio["audio"]) # Português (transcrição) - transcribe

{'text': ' Posso depositar dinheiro?'}

In [31]:
reconhecedor(audio["audio"]) # Inglês (tradução) - translate

{'text': ' Can I deposit money?'}

In [32]:
import sounddevice as sd

duracao = 10
taxa_amostragem = 16000
tamanho_vetor = int(duracao * taxa_amostragem)

gravacao = sd.rec(tamanho_vetor, samplerate=taxa_amostragem, channels=1).ravel()
sd.wait()

PortAudioError: Error querying device -1

In [33]:
reconhecedor({"raw": gravacao.ravel(), "sampling_rate": taxa_amostragem})

NameError: name 'gravacao' is not defined